In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f transport-all-comments.csv.xz || wget  https://github.com/heiseacademy/ml-python/raw/main/06-ml-projekt/transport-all-comments.csv.xz")

In [ ]:
import pandas as pd

posts = pd.read_csv("transport-all-comments.csv.xz", parse_dates=["created_utc"])

In [ ]:
tesla = posts[posts["text"].str.contains("tesla")].copy()
len(tesla)

In [ ]:
musk = posts[posts["text"].str.contains("musk")].copy()
len(musk)

In [ ]:
!pip install torch

In [ ]:
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print("Using GPU %s" % torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Using CPU :-(")

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer

model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

In [ ]:
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax


# das Modell muss zum Tokenizer passen!
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    output_attentions = False,
    output_hidden_states = False # wir benötigen keine Embeddings
)
# hier evtl. model.cuda() einsetzen
model.cpu()

In [ ]:
from tqdm.auto import tqdm
import numpy as np

def calculate_sentiment(df):
    # in scores kommen die Ergebnisse rein
    scores = []
    
    # die Schleife nutzt 100er Batches
    for i in tqdm(range((len(df)-1)//100 + 1)):
        # wichtige interne Datenstrukturen
        input_ids = []
        attention_masks = []
        # damit iterierst du über 100 Datensätze im DataFrame
        for t in df[i*100:(i+1)*100]["text"].map(str).values:
            # die Texte tokenisieren
            encoded_dict = tokenizer.encode_plus(
                                t,
                                add_special_tokens = True,    # '[CLS]' und '[SEP]'
                                max_length = 64,
                                truncation = True,
                                padding='max_length',
                                return_attention_mask = True,  # Attention-Masks erzeugen
                                return_tensors = 'pt',         # pytorch-Tensoren als Ergebnis
                           )
            # interne Strukturen befüllen
            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])

        # Jetzt hast du die input_ids und attention_masks für den Batch bestimmt
        # nun musst du sie noch in Tensoren wandeln
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)        

        # Du willst das Modell nur auswerten, nicht trainieren, daher ist kein Gradient notwendig
        with torch.no_grad():
            # Auswertung durchführen (dieser Schritt dauert!)
            res = model(input_ids.to(device), attention_mask=attention_masks.to(device))
            # res[0] enthält die Ergebnisse, das .cpu().detach() ist für GPUs notwendig
            for r in res[0].cpu().detach().numpy():
                # du speicherst in Scores die softmax-Werte für alle Sentiment-Ergebnisse,
                # also im Prinzip die Wahrhscheinlichkeit für Sentiment 1, 2, 3, 4 und 5
                scores.append(list(softmax(r)))
    
    # jetzt überträgst du die Sentimentwerte en bloc in den DataFrame
    df["s1"] = df["s2"] = df["s3"] = df["s4"] = df["s5"] = None
    df[["s1", "s2", "s3", "s4", "s5"]] = scores
    
    # das ist das "wahrscheinlichste" Sentiment
    df["sentiment"] = [np.argmax(s)+1 for s in df[["s1", "s2", "s3", "s4", "s5"]].values]
    
    # und hier berechnest du den Erwartungswert
    df["sentiment_avg"] = [s[0] + 2*s[1] + 3*s[2] + 4*s[3] + 5*s[4] 
                                for s in df[["s1", "s2", "s3", "s4", "s5"]].values]
    
    # die Varianz gibt die einen Eindruck über die Verlässlichkeit...
    df["sentiment_var"] = [(s[0] + 2*2*s[1] + 3*3*s[2] + 4*4*s[3] + 5*5*s[4]) - 
                               (s[0] + 2*s[1] + 3*s[2] + 4*s[3] + 5*s[4])**2
                                  for s in df[["s1", "s2", "s3", "s4", "s5"]].values]
    
    # ... genau wie die Standardabweichung
    df["sentiment_dev"] = np.sqrt(df["sentiment_var"])
    
    return df

In [ ]:
calculate_sentiment(tesla)

In [ ]:
calculate_sentiment(musk)

In [ ]:
tesla

In [ ]:
ts = tesla.set_index("created_utc")[["sentiment"]].resample("M").mean()
ms = musk.set_index("created_utc")[["sentiment"]].resample("M").mean()

In [ ]:
cs = ts.merge(ms, how="outer", left_index=True, right_index=True)

In [ ]:
cs = cs[["sentiment_x", "sentiment_y"]].rename(columns={"sentiment_x": "sentiment_tesla", 
                                                        "sentiment_y": "sentiment_musk"})
cs.plot(figsize=(16,9))

In [ ]:
if ON_COLAB:
    os.system("test -f tesla-stock.csv || wget  https://github.com/heiseacademy/ml-python/raw/main/06-ml-projekt/tesla-stock.csv")

In [ ]:
# source: https://www.nasdaq.com/market-activity/stocks/tsla/historical
stock = pd.read_csv("tesla-stock.csv", parse_dates=["Date"])

In [ ]:
stock.set_index("Date")["Close/Last"].plot()

In [ ]:
stock_scale = stock.set_index("Date").resample("M").mean()
stock_scale["stock_value"] = stock_scale["Close/Last"] / 100

In [ ]:
css = cs.merge(stock_scale, how="outer", left_index=True, right_index=True)

In [ ]:
css[["sentiment_tesla", "sentiment_musk", "stock_value"]].plot(figsize=(16, 9))

In [ ]:
css[["sentiment_tesla", "sentiment_musk", "stock_value"]].plot(figsize=(16, 9), logy=True)